In [1]:
import os
import sys
sys.path.append("../ECCOv4-py")
import ecco_v4_py as ecco
from ecco_download import *
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
home = "~/Documents/Github/OTP/ECCO"
shortname = "ECCO_L4_TEMP_SALINITY_05DEG_MONTHLY_V4R4"

ecco_podaac_download(ShortName=shortname,
                     StartDate="2014-08-01", EndDate="2020-12-31", download_root_dir="data",
                     n_workers=6, force_redownload=True)

created download directory data/ECCO_L4_TEMP_SALINITY_05DEG_MONTHLY_V4R4

Total number of matching granules: 41
DL Progress: 100%|#########################| 41/41 [00:40<00:00,  1.02it/s]

total downloaded: 841.77 Mb
avg download speed: 20.97 Mb/s
Time spent = 40.14610481262207 seconds


In [3]:
ecco = xr.open_mfdataset(f"{home}/data/{shortname}/*.nc", parallel=True, 
                         data_vars="minimal", coords="minimal", compat="override")
osnap = xr.open_dataset("../OSNAP/OSNAP_Gridded_TSV_201408_202006_2023.nc")

# need to be careful to match depth, first.
ecco = ecco.sel(Z=-15)
osnap = osnap.sel(DEPTH=15.)

[-5.000000e+00 -1.500000e+01 -2.500000e+01 -3.500000e+01 -4.500000e+01
 -5.500000e+01 -6.500000e+01 -7.500500e+01 -8.502500e+01 -9.509500e+01
 -1.053100e+02 -1.158700e+02 -1.271500e+02 -1.397400e+02 -1.544700e+02
 -1.724000e+02 -1.947350e+02 -2.227100e+02 -2.574700e+02 -2.999300e+02
 -3.506800e+02 -4.099300e+02 -4.774700e+02 -5.527100e+02 -6.347350e+02
 -7.224000e+02 -8.144700e+02 -9.097400e+02 -1.007155e+03 -1.105905e+03
 -1.205535e+03 -1.306205e+03 -1.409150e+03 -1.517095e+03 -1.634175e+03
 -1.765135e+03 -1.914150e+03 -2.084035e+03 -2.276225e+03 -2.491250e+03
 -2.729250e+03 -2.990250e+03 -3.274250e+03 -3.581250e+03 -3.911250e+03
 -4.264250e+03 -4.640250e+03 -5.039250e+03 -5.461250e+03 -5.906250e+03]
[  15.   35.   55.   75.   95.  115.  135.  155.  175.  195.  215.  235.
  255.  275.  295.  315.  335.  355.  375.  395.  415.  435.  455.  475.
  495.  515.  535.  555.  575.  595.  615.  635.  655.  675.  695.  715.
  735.  755.  775.  795.  815.  835.  855.  875.  895.  915.  935.  95

In [5]:
# find the bounds of the OSNAP array (these are the locations we can use for validation against OSNAP)

min_lat = osnap["LATITUDE"].min().values
max_lat = osnap["LATITUDE"].max().values
min_lon = osnap["LONGITUDE"].min().values
max_lon = osnap["LONGITUDE"].max().values

# osnap slice to reduce size.
ecco = ecco.sel(latitude=slice(min_lat, max_lat),
                longitude=slice(min_lon, max_lon))

# we'll take some lat/lon on the array.
lat = ecco["latitude"].median().values
lon = ecco["longitude"].median().values

ecco_subset = ecco.sel(latitude=slice(min_lat, max_lat), longitude=slice(min_lon, max_lon))
osnap_subset = osnap.sortby(["LATITUDE", "LONGITUDE"]).sel(LATITUDE=slice(min_lat, max_lat), LONGITUDE=slice(min_lon, max_lon))

In [6]:
ecco_subset = ecco_subset.assign_coords(month=ecco_subset["time"].dt.month)
osnap_subset = osnap_subset.assign_coords(month=osnap_subset["TIME"].dt.month)

# Find the start and end of the common time range
common_start = max(ecco_subset["time"].min(), osnap_subset["TIME"].min())
common_end = min(ecco_subset["time"].max(), osnap_subset["TIME"].max())

# Subset the datasets to the common time range
ecco_subset = ecco_subset.sel(time=slice(common_start, common_end))
osnap_subset = osnap_subset.sel(TIME=slice(common_start, common_end))

# now the dates are only out by half a month (possibly just due to a convention difference) so when plotting I used the ecco time index.


print(ecco_subset["THETA"].values)
print(osnap_subset["TEMP"].values)

merged = xr.merge([ecco_subset, osnap_subset], compat="override", join="outer")

[[[        nan         nan  9.870409   ...         nan 16.852154
   16.852154  ]
  [        nan         nan  9.51067    ...         nan         nan
           nan]
  [        nan         nan  8.918689   ...         nan         nan
           nan]
  ...
  [ 9.300205    9.300205    9.708469   ... 13.569895   13.852292
   13.852292  ]
  [ 9.298086    9.298086    9.716058   ... 13.115824   13.408332
   13.408332  ]
  [ 9.298392    9.298392    9.6966305  ... 13.115824   13.408332
   13.408332  ]]

 [[        nan         nan  7.857473   ...         nan 16.131998
   16.131998  ]
  [        nan         nan  7.3780518  ...         nan         nan
           nan]
  [        nan         nan  6.8414865  ...         nan         nan
           nan]
  ...
  [ 8.011893    8.011893    8.397915   ... 13.0195675  13.221812
   13.221812  ]
  [ 8.1504135   8.1504135   8.482783   ... 12.471053   12.734896
   12.734896  ]
  [ 8.243308    8.243308    8.51621    ... 12.471053   12.734896
   12.734896  ]]

 [[ 

In [7]:
merged

<xarray.Dataset>
Dimensions:         (time: 41, latitude: 16, longitude: 102, nv: 2, TIME: 41,
                     LATITUDE: 256, LONGITUDE: 256)
Coordinates: (12/13)
  * time            (time) datetime64[ns] 2014-08-16T12:00:00 ... 2017-12-16T...
    Z               float32 -15.0
  * latitude        (latitude) float32 52.25 52.75 53.25 ... 58.75 59.25 59.75
  * longitude       (longitude) float32 -56.75 -56.25 -55.75 ... -6.75 -6.25
    time_bnds       (time, nv) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    latitude_bnds   (latitude, nv) float32 dask.array<chunksize=(16, 2), meta=np.ndarray>
    ...              ...
    Z_bnds          (nv) float32 dask.array<chunksize=(2,), meta=np.ndarray>
    month           (time) int64 8 9 10 11 12 1 2 3 4 5 ... 4 5 6 7 8 9 10 11 12
  * TIME            (TIME) datetime64[ns] 2014-08-01T12:00:00 ... 2017-12-01T...
  * LATITUDE        (LATITUDE) float32 52.02 52.05 52.08 ... 60.19 60.2 60.23
  * LONGITUDE       (LONGITUDE) float32 -56.88 -56.63 -56.39 ... -6.358 -6.119
    DEPTH           float32 15.0
Dimensions without coordinates: nv
Data variables:
    THETA           (time, latitude, longitude) float32 dask.array<chunksize=(1, 16, 102), meta=np.ndarray>
    SALT            (time, latitude, longitude) float32 dask.array<chunksize=(1, 16, 102), meta=np.ndarray>
    VELO            (TIME, LONGITUDE) float32 ...
    TEMP            (TIME, LONGITUDE) float32 nan nan nan nan ... 10.29 nan nan
    SAL             (TIME, LONGITUDE) float32 ...
Attributes: (12/62)
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Ian Fenty and Ou Wang
    cdm_data_type:                   Grid
    comment:                         Fields provided on a regular lat-lon gri...
    Conventions:                     CF-1.8, ACDD-1.3
    coordinates_comment:             Note: the global 'coordinates' attribute...
    ...                              ...
    time_coverage_duration:          P1M
    time_coverage_end:               2014-02-01T00:00:00
    time_coverage_resolution:        P1M
    time_coverage_start:             2014-01-01T00:00:00
    title:                           ECCO Ocean Temperature and Salinity - Mo...
    uuid:                            848d81f8-4159-11eb-a71e-0cc47a3f57e1